# Lab 2: BC4 and your first neural network

In today's lab session we will 

1. [Build a Shallow Convolutional Neural Network (CNN)](#Building-a-Shallow-Convolutional-Neural-Network)
2. [Train that network on CIFAR-10](#Training-on-CIFAR-10)

---
# Building a Shallow Convolutional Neural Network

We'll be building a shallow Convolutional Neural Network (CNN) of two layers.

We'll be making heavy use of pytorch's layers today, defined in the [`torch.nn`](https://pytorch.org/docs/1.2.0/nn.html) module. 

**Task:** Open the documentation for the fully connected layer ([`nn.Linear`](https://pytorch.org/docs/1.2.0/nn.html#linear) and 2D convolutional layer ([`nn.Conv2d`](https://pytorch.org/docs/1.2.0/nn.html#conv2d)), you'll need these later.

Optimizers are defined in [`torch.optim`](https://pytorch.org/docs/1.2.0/optim.html). We'll use the [`SGD`](https://pytorch.org/docs/1.2.0/optim.html#torch.optim.SGD) optimizer like we used in the first lab.

[Loss functions](https://pytorch.org/docs/1.2.0/nn.html#loss-functions) are also part of [`torch.nn`](https://pytorch.org/docs/1.2.0/nn.html#loss-functions) module. Also find the documentation for `nn.CrossEntropyLoss` (we used this last week) -- you'll need this later.

We will implement the following architecture, as described in your practical intro slides.

<img alt="CNN Architecture diagram" src="./media/cnn-ex-8.png" style="max-height: 300px;">

This diagram is drawn in the style put forward in the [AlexNet paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) where inputs/outputs are visualised as 3D volumes and layers are implicit between the inputs/outputs with only the receptive fields drawn (highlighted in orange, pink, and blue in the figure above).

The output shapes of each layer are listed in the table below

| Layer  | Output shape ($C \times H \times W$) |
|--------|------------------------------|
| Input  | $3 \times 32 \times 32$      |
| Conv1  | $32 \times 32 \times 32$     |
| Pool1  | $32 \times 16 \times 16$     |
| Conv2  | $64\times 16 \times 16$     |
| Pool2  | $64 \times 8 \times 8$       |
| FC1    | $1024$                       |
| FC2    | $10$                         |

Our network is designed to operate on images from CIFAR-10, a dataset containing 60,000 RGB images, each 32 $\times$ 32 in resolution, split into 50,000 images for training and 10,000 images for testing. 

There are 10 classes with 6,000 examples per class. Some examples of each class can be seen in the diagram below

<img alt="CIFAR-10 examples" src="./media/cifar10.png" style="max-height: 500px;" />


We've provided you with a boilerplate script `train_cifar.py` to get you started. Download the code to your laptop by cloning the git repository  to your laptop:

```console
$ git clone https://github.com/COMSM0045-Applied-Deep-Learning/labsheets.git
```

If you don't have git, then download a zip copy via the green button in the top right of https://github.com/COMSM0045-Applied-Deep-Learning/labsheets.


The code provided is in `lab-2-cnns/lab2-code/`. There are two files:
- `train_cifar.py`: This contains the code that you will need to edit in this lab
- `train_cifar.sh`: This is a slurm job script which you will run on BC4 to train and evaluate your network when you aren't using interactive sessions.

We'll draw the architecture of the CNN bit by bit, accompanying it by code to show you how to implement the first few layers. We'll leave you to implement the rest. 

First up is the input to the network, this is a single input image drawn as a 3D volume:

<img alt="Input shape" src="./media/cnn-ex-1.png" style="max-height: 200px;">

In PyTorch, our network definition is split into two parts: The first part allocates the memory for the parameterized layers and takes place in the constructor, the second part defines the forward pass defining how the input data flows through the layers.

In this snippet of code from `train_cifar.py`, we define the bare skeleton of the CNN object. It has a few constructor arguments that define the shape of the input which we store in an `ImageShape` data structure.

```python
class ImageShape(NamedTuple):
    height: int
    width: int
    channels: int

        
class CNN(nn.Module):
    def __init__(self,
                 height: int = 32,
                 width: int = 32,
                 channels: int = 3,
                 class_count: int = 10):
        self.input_shape = ImageShape(height, width, channels)
        self.class_count = class_count
            ...
```

Our first layer sits in between the input tensor and the output tensor. One of the convolutional filter's receptive field is visualised by the orange cube in the input. Once the filter has been convolved with the input at one position, it produces a single value in the output tensor, indicated by the tip of the orange pyramid. The depth (horizontal) of the output tensor indicates the number of convolutional filters of the layer, in this case that is 32, i.e. there are 32 different orange cubes (different filter weights) convolved with the input.

<img alt="First conv layer" src="./media/cnn-ex-2.png" style="max-height: 200px;">

We define the convolutional layer as an attribute in the constructor, and pass the `images` through it during the `forward` pass.

```python
from torch.nn import functional as F


class CNN(nn.Module):
    def __init__(self,
                 height: int = 32,
                 width: int = 32,
                 channels: int = 3,
                 class_count: int = 10):
        ...
        self.conv1 = nn.Conv2d(
            in_channels=self.input_shape.channels,
            out_channels=32,
            kernel_size=(5, 5),
            padding=(3, 3),
        )
        self.initialise_layer(self.conv1)
        ...
        
    def forward(images: torch.Tensor) -> torch.Tensor:
        ...
        x = F.relu(self.conv1(images))
        ...
        
        
    @staticmethod
    def initialise_layer(layer):
        if hasattr(layer, "bias"):
            nn.init.zeros_(layer.bias)
        if hasattr(layer, "weight"):
            nn.init.kaiming_normal_(layer.weight)
```

Note that we apply the ReLU function in the forward pass, this is defined in the `torch.nn.functional` module (traditionally imported with the alias `F`) .

We have defined a method `initialise_layer` that initialises the layer's parameters. The `bias` weight is initialised with zeros, and for the `weight` attribute we use [`kaiming_normal_`](https://pytorch.org/docs/1.2.0/nn.init.html#torch.nn.init.kaiming_uniform_) to initialise the weight matrix with values from $\mathcal{N}(0, \sigma^2)$ where $\sigma$ is dependent upon the number of inputs to the layer.

Next we halve the spatial dimensions of the output of the convolutional layer by 'pooling' its output by placing a $2 \times 2$ grid at each position in the input and taking the max value within this grid. The purple grid in the figure slides across the full tensor. Note that the channel dimension stays the same--that's because pooling is applied for *each* channel dimension.

<img alt="First pooling layer" src="./media/cnn-ex-3.png" style="max-height: 200px;">

```python
from torch.nn import functional as F


class CNN(nn.Module):
    def __init__(self,
                 height: int = 32,
                 width: int = 32,
                 channels: int = 3,
                 class_count: int = 10):
        ...
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        ...

    def forward(images: torch.Tensor) -> torch.Tensor:
        ...
        x = F.relu(self.conv1(images))
        x = self.pool1(x)
        ...
```

Let's copy the code over to BC4 and get an interactive session on a GPU node so you can test your code as you implement the subsequent layers.

**BEWARE**: the code won't yet run without crashing as you need to implement the rest of the network and training process; what we've provided is just skeleton code!


Copy `train_cifar.py` file to BC4 **every time you update it on your machine**.

**Linux/MacOS/WSL users:**

Copy the files over to BC4
```console
$ cd path/to/files
$ scp -r lab2-code bc4-external:/path/to/destination/
```
(If you didn't set up a `bc4-external` alias in `~/.ssh/config` last week we suggest you do this now. If not you can first copy the file to snowy with: `scp -r lab2-code <username>@snowy.cs.bris.ac.uk:~/` and then copying the file to blue crystal by first sshing into snowy `ssh <username>@snowy.cs.bris.ac.uk` and then copying the file from snowy to bluecrystal:  `scp -r lab2-code <username>@bc4login.acrc.bris.ac.uk:/path/to/destination/`.)

In a **different** console, login to BC4 and get an interactive session on a GPU node

```console
$ ssh <username>@bc4login.acrc.bris.ac.uk  # connect to a BC4 login node
[bc4] $ cd ~/lab2-code
[bc4] $ module load languages/anaconda3/2021-3.8.8-cuda-11.1-pytorch
[bc4] $ srun --job-name lab1 --partition teach_gpu --nodes 1 --gres gpu:1 --time 0:20:00 --mem 4GB python train_cifar.py
```

It's your turn now to implement the remaining CNN layers. We're going to help you approach this in a step-by-step manner so you can verify your progress as you add each layer. To accomplish this, we'll print out the output shape of the network as we add each layer. This is useful because it shows you the output shape of the last layer you implemented, and consequently the input shape of the next one you need to implement.

**Task 1:** Within the training loop in the `Trainer.train` method, compute the model's forward pass using `self.model` on the `batch`. `batch` contains a $N \times 3 \times 32 \times 32$ batch of CIFAR-10 images. Assign the result of the forward pass to a local variable named `output`. Finally, print the output shape, and quit the program.

```python
## TASK 1: Compute the forward pass of the model, print the output shape
##         and quit the program
output = self.model.forward(batch)
print(output.shape)
import sys; sys.exit(1)
```            

Run your code with the command `python train_cifar.py`. You're program should print the output shape of the first conv layer:

```python
torch.Size([128, 32, 16, 16])
```

The data layout is in `NCHW` format, where 

- `N` is the batch size
- `C` is the channel depth
- `H` is the height
- `W` is the width

The batch size is 128 in this case, the spatial dimensions are $16 \times 16$, and the channel depth is 32.

**Task 2:** Now add the second convolutional layer, it has a $5 \times 5$ kernel, 
Don't forget to pass the output through `relu` in the `forward` function. Run the code and verify that the output shape against the diagram.

<img alt="Second conv layer" src="./media/cnn-ex-4.png" style="max-height: 200px;">

You'll need to add code below the following comments in your script
```python
## TASK 2-1: Define the second convolutional layer and initialise its parameters.
## Hint: copy the code for conv1, changing the name as well as the arguments for in_channels and out_channels. Also remember to initialise this layer!
``` 
and
```python
## TASK 2-2: Pass x through the second convolutional layer
## Hint: Don't forget to pass it through a relu after the convolution. 
```

**Task 3:** Next add the second pooling layer. Again you need to define this layer at `## Task 3-1` before calling this layer at `## Task 3-2`.  Run the code and verify the output shape.

<img alt="Second pooling layer" src="./media/cnn-ex-5.png" style="max-height: 200px;">

**Task 4:** Flatten the tensor produced by the second pooling layer from $8 \times 8 \times 64$ to $4096$, use [`torch.flatten`](https://pytorch.org/docs/1.2.0/torch.html) for this; take special note of the `start_dim` kwarg. You need to set this to 1 (default is 0) to avoid flattening the whole batch. All the functions in Pytorch expect a tensor of shape $N \times \ldots$ where $N$ is the batch size. By setting `start_dim` to 1, we flatten each image, not the whole batch. Run the code and check your output is a 2D tensor, first dimension is the batch size, and the second should be 4096.

<img alt="Flattened convolutional features" src="./media/cnn-ex-6.png" style="max-height: 300px;">

**Task 5:** Now take the flattened features and pass them through a fully connected layer (a.k.a a [`Linear`](https://pytorch.org/docs/1.2.0/nn.html#torch.nn.Linear) layer in PyTorch) mapping the $4096$ feature to $1024$. Copy the code

```python
## TASK 5-1: Define the first FC layer and initialise its parameters
self.fc1 = nn.Linear(4096, 1024)
self.initialise_layer(self.fc1)
```
Now add code to use this layer in the forward pass under (don't forget the ReLU activation function after the fully connected layer)
```python
## TASK 5-2: Pass x through the first fully connected layer
```
Run the code and check the output shape is `(128, 1024)`.

<img alt="First FC layer" src="./media/cnn-ex-7.png" style="max-height: 300px;">

**Task 6:** Add the final fully connected layer that maps from the $1024$ feature to the number of classes. This layer produces our [*logits*](https://developers.google.com/machine-learning/glossary/#logits), the unbounded class scores (**do NOT** use ReLU after this layer). Run the code and check the output shape is `(128, 10)`.

<img alt="Final FC layer" src="./media/cnn-ex-8.png" style="max-height: 300px;">

---
# Training on CIFAR-10

Now that you've defined your network we can go ahead and train it. To do so we need a loss function and an optimizer. For our loss function we'll use softmax cross entropy, the standard loss function for single-label classification tasks. For our optimizer we'll use Stochastic Gradient Descent (SGD). 

**Task 7:** Rename the `output` variable in he training loop of the `Trainer.train` method where you store you model output to `logits` as the model now produces a logit vector of class scores; the subsequent code in `Trainer.train` depends on this variable. 

Also remove the code you previously wrote to print the output and exit
```python
print(output.shape)
import sys; sys.exit(1)
```

**Task 8:** In the `main(args)` function, replace
```python
## TASK 8: Redefine the criterion to be softmax cross entropy
criterion = lambda logits, labels: torch.tensor(0)
```
with
```python
## TASK 8: Redefine the criterion to be softmax cross entropy
criterion = nn.CrossEntropyLoss()
```
This defines the softmax cross entropy loss [`nn.CrossEntropyLoss`](https://pytorch.org/docs/1.2.0/nn.html#torch.nn.CrossEntropyLoss). You will need to remove the dummy loss code `lambda logits, labels: torch.tensor(0)`, which was only there so you could run your code in previous steps to check your progress so far.

**Task 9:** Back in the training loop, replace the dummy loss function
```python
## TASK 9: Compute the loss using self.criterion and
##         store it in a variable called `loss`
loss = torch.tensor(0)
```
with your loss computed using `self.criterion`. This takes two arguments: the `logits` and the local variable `labels` (a 1D vector of length $N$ containing the labels corresponding to each example in the batch).

Run the code and check the loss printed out matches chance performance. Accuracy chance is 10% for randomly selecting one out of 10 classes. You don't need to run the code until completion, you can kill it after you've checked performance for a few batches by pressing `<Ctrl-C>`.

Q. Why does the accuracy change over different iterations? 

A. Because in each iteration, the network predicts the classes for a different batch of examples (a set of 128 randomly selected images).

We have **not** yet trained our model. This was simply the forward pass.
 
**Task 10:** Compute the backward pass (this populates the gradient buffers of your network parameters) by calling `backward` on your `loss` variable (i.e. `loss.backward()`). 

**Task 11:** Initialise the variable named `optimizer` in the `main` function to a [`torch.optim.SGD`](https://pytorch.org/docs/1.2.0/optim.html#torch.optim.SGD) object using the [parameters of your model](https://pytorch.org/docs/1.2.0/nn.html#torch.nn.Module.parameters) and the learning rate stored in `args.learning_rate`.

**Task 12:** Update your network's parameters by calling [`self.optimizer.step()`](https://pytorch.org/docs/1.2.0/optim.html#torch.optim.Optimizer.step) and zero-out your gradient buffers using [`self.optimizer.zero_grad()`](https://pytorch.org/docs/1.2.0/optim.html#torch.optim.Optimizer.zero_grad) in `Trainer.train`. Run your code and check that your model's accuracy during training and testing increases and the loss decreases.

As you watch the model trained, you should see the accuracy increasing from chance (10%) to around **65%** by the end of epoch 20. Note the difference between your `batch accuracy` and your overall `accuracy` on the full test set.

```
epoch: [19], step: [390/391], batch loss: 0.76462, batch accuracy: 72.50, data load time: 0.00019, step time: 0.00363
validation loss: 0.95978, accuracy: 66.66
```

For example, at the end of my training, the batch loss for the last batch was 0.76 and the batch accuracy 72.5%.
However when testing the model on the full test set, the accuracy was 66.6% with the loss 0.96.

---

In addition to the metrics being printed to the console, the code we've provided also logs values to 
a tensorboard log directory. We're going to launch a tensorboard server on the GPU node and port forward this to your laptop so you can access it locally.

Each time you run your code, a new subdirectory containing that run's logs will be saved within the `logs` directory. You will see a bunch of subdirectorys already within `logs` when you were testing your code earlier as you built your network.


**Linux/mac/WSL** users:
```console
[bc4] $ PORT=$((($UID-6025) % 65274))  # compute a unique port and save to the PORT env var
[bc4] $ echo $PORT  # echo the PORT env var to the console so you can see what it is
<PORT>
[bc4] $ hostname -s  # get hostname.
<HOSTNAME>
[bc4] $ cd ~/lab2-code  # change the working directory to where the tensorboard logs are stored
[bc4] $ module load "languages/anaconda3/2021-3.8.8-cuda-11.1-pytorch"
[bc4] $ tensorboard --logdir logs --port "$PORT" --bind_all # run the tensorboard server on the port stored in the PORT env var
TensorBoard 1.14.0 at http://<HOSTNAME>.bc4.acrc.priv:<PORT>/ (Press CTRL+C to quit)
```

Open another terminal on your laptop and run an SSH process to forward the port `$PORT` on `<HOSTNAME>` to port 6006 on your machine where `<PORT>` should be replaced by the output of the `echo $PORT` command above and `<HOSTNAME>` with the hostname printed by `hostname -s`.

```console
$ ssh -N -L 6006:<HOSTNAME>:<PORT> bc4-external
```

Open up http://localhost:6006. Have a look at the loss and accuracy curves plotted for both training and test.

Make sure your tensorboard settings match those in the screenshot below:

![Tensorboard settings](./media/tensorboard-settings.png)

Your network should produce a similar result to the graphs shown below:

If you see multiple curves, it's because you've trained the model multiple times already. Each run will produce a new log folder within `logs`, clear all but the most recent one and rerun tensorboard.

![Expected accuracy](./media/expected-accuracy.png) ![Expected loss](./media/expected-loss.png)

**Task 13:** Run `python train_cifar.py --help` and investigate the hyperparameters that you can tweak. Pick one hyperparameter to change, run the code again and look at how the loss and accuracy curves differ.

# BC4 Batch mode

We provide you with the file `train_cifar.sh`, this is a file that slurm, BC4's job scheduling system, can execute to run your python script.
The script has the following contents,

```bash
#!/usr/bin/env bash
#SBATCH --job-name=lab2
#SBATCH --partition teach_gpu
#SBATCH --nodes=1
#SBATCH -o ./log_%j.out # STDOUT out
#SBATCH -e ./log_%j.err # STDERR out
#SBATCH --gres=gpu:1
#SBATCH --time=0:20:00
#SBATCH --mem 4GB

# get rid of any modules already loaded
module purge
# load in the module dependencies for this script
module load "languages/anaconda3/2021-3.8.8-cuda-11.1-pytorch"

python train_cifar.py
```

The lines starting with `#SBATCH` are instructions to the `sbatch` command which we use to submit the job.

Exit your interactive session and submit a job using the job script from the login node:

```console
[bc4] $ sbatch train_cifar.sh  # create a new job using the job script
[bc4] $ squeue --user $USER    # list the queued and running jobs owned by me
JOBID         PARTITION     NAME      USER     ST       TIME  NODES NODELIST(REASON)
2267572       gpu           train_ci  wp14832  PD       0:00      1 (Priority)
```

The last line above queries slurm asking which jobs in the queue belong to your user account. You can run the same `squeue` command to check the state of your job, it should change from `PD` to `R` when it is running, and then it'll disappear once complete.

Once your job has begun to run, it will produce an output file with the name `slurm-<jobid>.out` in the same directory from which you submitted it. In my case, my job had the ID `2267572`, so my output log was called `slurm-2267572.out`. This is written to interactively, so you can watch the results of your code as it runs by running `tail -f slurm-<jobid>.out` (press `<Ctrl>-c` to quit `tail`).